In [19]:
import os
os.getcwd()
os.chdir(r"src")
import matplotlib.pyplot as plt
import utils

In [10]:
# ---- utils libs ----
import datetime
import sys


# --- Import functions from utils.py ---
sys.path.insert(0,'..')
from utils import convertToSequenceParameters, read_pickle_dataset

# --- Import functions from preprocessing.py ---
sys.path.insert(0,'..')
from preprocessing import data_preprocessing

# --- Import functions from visualize.py ---
sys.path.insert(0,'../src/visualization')
from visualize import visualize_load_curve_dataset, visualize_load_curve_resampled, visualize_train_load_curve, visualize_test_load_curve, visualize_test_train_load_curve, visualize_report_preprocessing

# --- Import functions from build_model.py ---
sys.path.insert(0,'../src/models/')
from build_model import model

# --- Import functions from train_model.py ---
sys.path.insert(0,'../src/models/')
from train_model import train

# --- Import functions from eval_model.py ---
sys.path.insert(0,'../src/models/')
from eval_model import plot_train_val_loss, plot_reconstructed_base_load_curve, plot_activity_histogram, plot_activity_distibrution, confusion_matrix, evaluate

# --- Import functions from predict_model.py ---
sys.path.insert(0,'../src/models/')
from predict_model import X_train_predict, X_test_predict, compute_threshold, compute_train_mae_loss, compute_test_mae_loss, plot_train_mae_loss, plot_test_mae_loss, detect_anomaly_sequence, get_anomaly_df, get_df_predict, get_specific_anomaly_sequence, plot_anomaly_sequences, plot_anomaly_data_points

# --- Import functions from postprocessing.py ---
sys.path.insert(0,'..')
from postprocessing import data_postprocessing, plot_postprocessing_anomalies

In [13]:
# --- Define global variable ---
TIME_STEP = datetime.timedelta(minutes=1, seconds=30) # duration of a step in the resample dataset, originally 1 second
DURATION_TIME = datetime.timedelta(minutes=60) # duration of a sequence
OVERLAP_PERIOD_PERCENT = 0.8 # 0.5 <=> 50% overlapping
TIMEFRAMES = [(datetime.time(0,0,0), datetime.time(3,0,0))] # timeframes we consider as unactive
STRATEGY = "off_peak_time" # device, off_peak_time, label 
METHOD = "method_prediction_1" # method to choose for aggregating sequences
print("CONVERTING GLOBAL USER PARAMETERS...\n")
SEQUENCE_LENGTH, OVERLAP_PERIOD = convertToSequenceParameters(TIME_STEP, DURATION_TIME, OVERLAP_PERIOD_PERCENT)
print("\t\tValeur choisie \t Equivalent sequence\nTimestep : \t {}\nDuration :\t {} \t -->  {} \nOverlap :\t {} \t\t -->  {}".format(TIME_STEP, DURATION_TIME, SEQUENCE_LENGTH, OVERLAP_PERIOD_PERCENT, OVERLAP_PERIOD))



CONVERTING GLOBAL USER PARAMETERS...

		Valeur choisie 	 Equivalent sequence
Timestep : 	 0:01:30
Duration :	 1:00:00 	 -->  40 
Overlap :	 0.8 		 -->  32


In [47]:
import pandas as pd
import numpy as np
%reload_ext autoreload
import preprocessing
%autoreload 2
import utils

# <>
Objectif : développer une meilleure stratégie de train/test split